# Sample Notebook

...

## 1. Basic Setup

This section hides away the complexity of working with the Kernel. 

- Get necessary libs
- Ensure configurations for the following are set:
    - Azure OpenAI or OpenAI
    - Azure Search
    - Bing Search
    - Other Congnitive Services

### 1.a. Get necessary Nuget packages

Note: I'm after specific versions here instead of the latest versions. The lib is evolving so getting the latest might cause errors (Eg. Skills being moved to Plugins)

In [24]:
//#r "nuget: dotenv.net, *-*"
#r "nuget: dotenv.net, 3.1.2"

//#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel, 0.24.230918.1-preview"

//#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 0.24.230918.1-preview"

//#r "nuget: Microsoft.SemanticKernel.Skills.Core, *-*"
#r "nuget: Microsoft.SemanticKernel.Skills.Core, 0.24.230918.1-preview"

//#r "nuget: Microsoft.SemanticKernel.Skills.Web, *-*"
#r "nuget: Microsoft.SemanticKernel.Skills.Web, 0.24.230918.1-preview"

// Configurations
#r "nuget: Microsoft.Extensions.Configuration, *-*"
#r "nuget: Microsoft.Extensions.Configuration.FileExtensions, *-*"
#r "nuget: Microsoft.Extensions.Configuration.Json, *-*"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, *-*" 
#r "nuget: Microsoft.Extensions.Configuration.Binder, *-*"

Installed Packages dotenv.net, 3.1.2 Microsoft.Extensions.Configuration, 8.0.0-rc.1.23419.4 Microsoft.Extensions.Configuration.Binder, 8.0.0-rc.1.23419.4 Microsoft.Extensions.Configuration.EnvironmentVariables, 8.0.0-rc.1.23419.4 Microsoft.Extensions.Configuration.FileExtensions, 8.0.0-rc.1.23419.4 Microsoft.Extensions.Configuration.Json, 8.0.0-rc.1.23419.4 Microsoft.SemanticKernel, 0.24.230918.1-preview Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 0.24.230918.1-preview Microsoft.SemanticKernel.Skills.Core, 0.24.230918.1-preview Microsoft.SemanticKernel.Skills.Web, 0.24.230918.1-preview

### 1.b. Load the Kernel

In [25]:
#!import ../config/Setup.cs
#!import ../config/Utils.cs

var kernel = Setup.LoadKernel();
kernel.Display();

LoggerFactory
Memory
Skills
PromptTemplateEngine
HttpHandlerFactory


## 2. Explore the Kernel

### 2.a. Use the out-of-the-box kernel capibilities (a.k.a Skills or Plugins)

In this example we use the following capibilities:
- Use the `Bing Search` skill/plugin & lookup something from the web.
- Use the `Conversation` skill/plugin to 
    - Summarise findings from the search result.
    - Extract key topics
    - Extract action items if any
- Use the `Web File Download` skill/plugin & download a file

In [26]:
var webQuery = "What is the inflation rate in the US in September 2023?";

var inflationData = await kernel.Func("bing", "search").InvokeAsync(webQuery);
//Utils.Print(inflationData.Result);

var inflationSummary  = await kernel.Func("conversation", "SummarizeConversation").InvokeAsync(inflationData);
Utils.Print(inflationSummary.Result);

var inflationTopics  = await kernel.Func("conversation", "GetConversationTopics").InvokeAsync(inflationData);
Console.WriteLine(inflationTopics.Result);

var inflationActionItems  = await kernel.Func("conversation", "GetConversationActionItems").InvokeAsync(inflationData);
Utils.Print(inflationActionItems.Result);

The annual inflation rate in the United States increased to 3.7% for the 12
months ending in August 2023, according to data from the U.S. Labor Department.
This is a rise from the previous period's rate of 3.2%, and is slightly above
market forecasts of 3.6%. The Consumer Price Index for All Urban Consumers also
increased by 0.6% in August, and rose 3.7% over the last 12 months. The index
for all items excluding food and energy increased by 0.3% in August, and was up
4.3% over the year. Despite the rise in inflation, consumer expectations for
inflation fell in early September to the lowest levels in over two years,
indicating growing optimism about the economic outlook. The Federal Reserve
responded to the economic conditions with four 75 basis point rate hikes in the
second half of the year.

{
  "topics": [
    "Annual inflation rate",
    "United States",
    "U.S. Labor Department",
    "Consumer Price Index",
    "Urban Consumers",
    "Food and energy exclusion",
    "Consumer ex

### 2.b. Explore memories

Semantic Memory allows you to store the meaning of text that come from different data sources, and optionally to store the source text too.

These texts can be from the web, e-mail providers, chats, a database, or from your
local directory, and are hooked up to the Semantic Kernel through data source connectors.

In this example, I store 3 animals - dog, elephant & octopus. We then search for the relevant animal that `has tentacles`.

In [27]:
using System.Linq;

var db = "random-animals";

await kernel.Memory.SaveInformationAsync(
    db, id: "1", text: "Dog");
await kernel.Memory.SaveInformationAsync(
    db, id: "2", text: "Elephant");
await kernel.Memory.SaveInformationAsync(
    db, id: "3", text: "Octopus");

var memorySearch = "Has tentacles?";
await foreach (var item in kernel.Memory.SearchAsync(
    collection: db, query: memorySearch, limit: 1))
{
    Utils.Print($"{memorySearch} - {item.Metadata.Text}");
}

await kernel.Memory.RemoveAsync(db,"1");
await kernel.Memory.RemoveAsync(db,"2");
await kernel.Memory.RemoveAsync(db,"3");


Has tentacles? - Octopus



### 2.c. Large text memories

When dealing with large corpuses of data you can chunk & store them as memories.

In [28]:
// Read file & convert it into smaller chunks
var chunks = await Utils.GetChunksAsync("../../../data/intelligent-investor.txt");
Console.WriteLine($"Chunks: {chunks.Count()}");

var db = "intelligent-investor-book";
// Save chunks into memory
for (var i = 0; i < chunks.Count(); i++)
{
    var chunk = chunks[i];        
    // await kernel.Memory.SaveInformationAsync(
    //     db, // collection name
    //     chunk, // text to embed
    //     $"{db}-{i}", // id
    //     $"Dataset: {db} Chunk: {i}", // title or description
    //     i.ToString() // metadata
    // );

    await kernel.Memory.SaveReferenceAsync(
            collection: db,
            externalSourceName: "intelligent-investor",
            externalId: $"{db}-{i}",
            description: $"Dataset: {db} Chunk: {i}",
            text: chunk);

    Console.WriteLine($"Chunk {i} of {chunks.Count} saved to memory collection {db}");
}


Chunks: 46
Chunk 0 of 46 saved to memory collection intelligent-investor-book
Chunk 1 of 46 saved to memory collection intelligent-investor-book
Chunk 2 of 46 saved to memory collection intelligent-investor-book
Chunk 3 of 46 saved to memory collection intelligent-investor-book
Chunk 4 of 46 saved to memory collection intelligent-investor-book
Chunk 5 of 46 saved to memory collection intelligent-investor-book
Chunk 6 of 46 saved to memory collection intelligent-investor-book
Chunk 7 of 46 saved to memory collection intelligent-investor-book
Chunk 8 of 46 saved to memory collection intelligent-investor-book
Chunk 9 of 46 saved to memory collection intelligent-investor-book
Chunk 10 of 46 saved to memory collection intelligent-investor-book
Chunk 11 of 46 saved to memory collection intelligent-investor-book
Chunk 12 of 46 saved to memory collection intelligent-investor-book
Chunk 13 of 46 saved to memory collection intelligent-investor-book
Chunk 14 of 46 saved to memory collection intel

## 3. Your Own Skills or Plugins

### 3.1. Simple Plugin with 1 input

In [29]:
string prompt = """
Tell me a joke about {{$input}}.
""";

var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
        MaxTokens = 2000,
        Temperature = 0.2,
        TopP = 0.5,
    }
};

var promptTemplate = new PromptTemplate(
    prompt,                        // Prompt template defined in natural language
    promptConfig,                  // Prompt configuration
    kernel                         // SK instance
);

var randomPlugin = kernel.RegisterSemanticFunction("Random", "Joke", new SemanticFunctionConfig(promptConfig, promptTemplate));

var joke = await randomPlugin.InvokeAsync("chickens");

Console.WriteLine(joke);

Why don't chickens use Facebook?

Because they already have their own "pecking" order!


### 3.2 Explicitly call Chat Completions

In [30]:
using Microsoft.SemanticKernel.AI.ChatCompletion;
using Microsoft.SemanticKernel.TemplateEngine.Prompt;


var chatCompletionService = kernel.GetService<IChatCompletion>();
var context = kernel.CreateNewContext();
var promptRenderer = new PromptTemplateEngine();


// 1. Set system message
string systemMessage = """
You are an AI assistant. Your name is {{ $name }}.
""";

// 2. Set user message
string userMessage = """
Hello there, I'm Andy.
""";

// 3. Set context variables
context.Variables["name"] = "Jane";

// 4. Create new chat history
var chatHistory = chatCompletionService.CreateNewChat(await promptRenderer.RenderAsync(systemMessage, context));
chatHistory.AddUserMessage(await promptRenderer.RenderAsync(userMessage, context));

// 5. Generate response
string answer = await chatCompletionService.GenerateMessageAsync(chatHistory, new ChatRequestSettings
{
    MaxTokens = 200,
    Temperature = 0.2,
    TopP = 0.5,
    StopSequences = new[] { "\n" }
});

// 6. Display response
chatHistory.Display();
answer.Display();



index value 0 Microsoft.SemanticKernel.AI.ChatCompletion.ChatHistory+ChatMessage Role system Label system Content You are an AI assistant. Your name is Jane. 1 Microsoft.SemanticKernel.AI.ChatCompletion.ChatHistory+ChatMessage Role user Label user Content Hello there, I'm Andy.

Hello Andy, nice to meet you! How can I assist you today?